In [2]:
from gensim import models

# takes a lilbit
# model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  
model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  

In [3]:
# imports and permanent stuff

import requests
import json
import pickle
import os
import numpy as np
import time

list_of_scrape_times = []
list_of_types = ['text','log','contact','calender','file','tweets']

In [4]:
## ELEMENTWISE FEATURIZING FUNCTION DEFINITIONS

# takes in element of list_of_jsons and scrape date, spits out text frequency 
def textFreq(texts, scrapedate):
    
    a = texts.json()
    
    seconds_intwo_weeks = 1209600;
    
    two_weeks_prior = scrapedate - seconds_intwo_weeks
    
    text_date = 0
    saved_index = 0
    
    for i in range(0,len(a)):
        text_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
        if (two_weeks_prior < text_date):
            saved_index = len(a) - (i+1)
            break
    
    return float(saved_index+1)/14
    
    
# takes in element of list_of_jsons(resp object) and scrape date, spits out call frequency 
def callFreq(calls, scrapedate):
    
    a = calls.json()
    
    seconds_intwo_weeks = 1209600;
    
    two_weeks_prior = scrapedate - seconds_intwo_weeks
    
    call_date = 0
    saved_index = 0
    
    for i in range(0,len(a)):
        call_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
        if (two_weeks_prior < call_date):
            saved_index = len(a) - (i+1)
            break
    
    return float(saved_index+1)/14


## WORD2VEC BB

# input is tweet, returns vector embedding of entire tweet.
# eg: a3.json()[0]
def tweetToEmbedding(tweet):
    
    q = json.loads(tweet)['text'].split()
    
    sumVector = np.zeros((300,))

    for i in range(0,len(q)):
        try:
            sumVector += model[q[i]]
        except KeyError:
            pass
    
    return sumVector

#input is tweets, return master vector
def embeddingToMastersum(tweets):
    
    masterSum = np.zeros((300,))
    
    for i in range(0,10):
        try:
            masterSum += tweetToEmbedding(tweets.json()[i])
        except KeyError:
            pass

    return masterSum

#input is tweets, return follocount
def followerCount(tweets):

    followerCount = 0
    
    try:
        followerCount = json.loads(tweets.json()[0])['user']['followers_count']
    except KeyError:
        pass

    return followerCount

#input is tweets, return friends (who user follows)
def followingCount(tweets):

    followingCount = 0
    
    try:
        followingCount = json.loads(tweets.json()[0])['user']['friends_count']
    except KeyError:
        pass

    return followingCount

#input is tweets, return avg likes per post for the last 2 weeks
def twitterLikeFreq(tweets, scrapedate):

    a = tweets.json()
    
    seconds_intwo_weeks = 1209600;
    
    two_weeks_prior = scrapedate - seconds_intwo_weeks
    
    tweet_date = 0
    saved_index = 0
    
    for i in range(0,len(a)):
        
        utc = json.loads(a[len(a)-(i+1)])['created_at']
        
        tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))
        
        if (two_weeks_prior < tweet_date):
            saved_index = len(a) - (i+1)
            break
    
    sum_of_likes = 0
    
    for i in range(0, saved_index):
        sum_of_likes += json.loads(tweets.json()[i])['favorite_count']
    
    
    return sum_of_likes/14
    
#input is tweets, return avg retweets per post for the last 2 weeks
def twitterRetweetFreq(tweets, scrapedate):

    a = tweets.json()
    
    seconds_intwo_weeks = 1209600;
    
    two_weeks_prior = scrapedate - seconds_intwo_weeks
    
    tweet_date = 0
    saved_index = 0
    
    for i in range(0,len(a)):
        
        utc = json.loads(a[len(a)-(i+1)])['created_at']
        
        tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))
        
        if (two_weeks_prior < tweet_date):
            saved_index = len(a) - (i+1)
            break
    
    sum_of_retweets = 0
    
    for i in range(0, saved_index):
        sum_of_likes += json.loads(tweets.json()[i])['retweet_count']
    
    
    return sum_of_retweets/14

In [5]:
r = requests.get('http://depressionmqp.wpi.edu:8080/getids')
list_of_idtime = r.json()
list_of_ids = []
list_of_times = []

for i in range(0,len(list_of_idtime)):
    list_of_ids.append( list_of_idtime[i]['id'].encode('ascii','ignore') )
    
for i in range(0,len(list_of_idtime)):
    list_of_times.append( list_of_idtime[i]['date'] )

#id1 = list_of_ids[0].encode('ascii','ignore')

number_cols = len(list_of_types)
number_rows = len(list_of_ids)
list_of_jsons = [[0] * number_cols for i in range(number_rows)]


### create directory for this particular scrape
timenow  = str(int(time.time())) # for temporal congruency
list_of_scrape_times.append(timenow)
os.mkdir('./datafor' + timenow)
filedir = './datafor' + timenow


for i in range(0,number_rows):
    for j in range(0,number_cols):
        temp = requests.get('http://depressionmqp.wpi.edu:8080/getdata?id=' + str(int(list_of_ids[0])) + '&type=' + list_of_types[j])
        
        pickle.dump(temp, open( filedir + "/DP" + str(int(list_of_ids[i])) +  list_of_types[j] + ".p", "wb" ))
        
        # loads it into memory. i will not use this for now
        # for architectural sanity
        # list_of_jsons[i][j] = temp

In [43]:
## DELETE DIRECTORY CELL

import shutil

def deletdatboi(nombre):
    shutil.rmtree('./datafor' + list_of_scrape_times[nombre])

#deletdatboi(1)

list_of_scrape_times


[]

In [1]:
num_of_people = len(list_of_ids)

featureVectorCF = np.zeros((num_of_people,1))
featureVectorTF = np.zeros((num_of_people,1))
featureVectorTW = np.zeros((num_of_people,300))
featureVectorFC = np.zeros((num_of_people,1))
featureVectorFC2 = np.zeros((num_of_people,1))
featureVectorTWL = np.zeros((num_of_people,1))
featureVectorTWRT = np.zeros((num_of_people,1))


for i in range(0, num_of_people):
    
    list_of_jsons[i][1]
    
    a1 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[i])) +  "log" + ".p", "rb" )) 
    a2 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[i])) +  "text" + ".p", "rb" )) 
    a3 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[i])) +  "tweets" + ".p", "rb" )) 

    
    featureVectorCF[i] = callFreq(a1, list_of_times[i])
    featureVectorTF[i] = textFreq(a2, list_of_times[i])
    featureVectorTW[i] = embeddingToMastersum(a3)
    featureVectorFC[i] = followerCount(a3)
    featureVectorFC2[i] = followingCount(a3)
    featureVectorTWL[i] = twitterLikeFreq(a3, list_of_times[i])
    featureVectorTWRT[i] = twitterRetweetFreq(a3, list_of_times[i])

    
featureMatrix = np.hstack((featureVectorCF,featureVectorTF, featureVectorTW, featureVectorFC, featureVectorTWL, featureVectorTWRT))

featureMatrix

NameError: name 'list_of_ids' is not defined

In [214]:


## access example
a3 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[0])) +  "tweets" + ".p", "rb" )) 
json.loads(a3.json()[0])






{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Nov 12 14:29:22 +0000 2017',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/usembassymanil…',
    'expanded_url': 'https://twitter.com/usembassymanila/status/929684767087378432',
    'indices': [11, 34],
    'url': 'https://t.co/TD0rYcWN8C'}],
  'user_mentions': []},
 'favorite_count': 40818,
 'favorited': False,
 'geo': None,
 'id': 929717762422988800,
 'id_str': '929717762422988801',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': True,
 'lang': 'en',
 'place': None,
 'possibly_sensitive': False,
 'quoted_status': {'contributors': None,
  'coordinates': None,
  'created_at': 'Sun Nov 12 12:18:15 +0000 2017',
  'entities': {'hashtags': [],
   'symbols': [],
   'urls': [{'display_url': 'twitter.com/i/web/status/9…',
     'expanded_url': 'https://t

In [143]:
w = embeddingToMastersum(a3)
w.reshape((-1, 1)).T.shape


(1, 300)

In [125]:
q = json.loads(a3.json()[1])['text'].split()
#a3.json()[0]

In [190]:
import datetime
dt = datetime.datetime.now()
utc = json.loads(a3.json()[0])['created_at']
r = utc.split()
r



['Sun', 'Nov', '12', '14:29:22', '+0000', '2017']

In [195]:
import time
print(int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y"))))

1510514962
